In [ ]:
%pip install langchain
%pip install langgraph
%pip install weaviate-client
%pip install ipywidgets
%pip install tqdm
%pip install -U langchain-ollama


In [5]:
import weaviate
import weaviate.classes.config as wcd


client = weaviate.WeaviateClient(
    connection_params=weaviate.connect.ConnectionParams.from_params(
        http_host="localhost",
        http_port=8080,
        http_secure=False,
        grpc_host="localhost",
        grpc_port=50051,
        grpc_secure=False,
    ),
    additional_config=weaviate.classes.init.AdditionalConfig(
        timeout=weaviate.classes.init.Timeout(init=30, query=60, insert=120),
    ),
    skip_init_checks=False
)
collection_name = "products"

In [2]:
ollama_api_endpoint = "http://host.docker.internal:11434"

if (client.collections.exists(collection_name)):
    client.collections.delete(collection_name)

client.collections.create(name=collection_name,
                          description="Contains all product for ecommerce website",
                          vectorizer_config=wcd.Configure.Vectorizer.text2vec_ollama(api_endpoint=ollama_api_endpoint, model="nomic-embed-text"),
                          generative_config=wcd.Configure.Generative.ollama(api_endpoint=ollama_api_endpoint, model="llama3.3"),
                          properties=[
                              wcd.Property(name="title", data_type=wcd.DataType.TEXT),
                              wcd.Property(name="category", data_type=wcd.DataType.TEXT),
                              wcd.Property(name="description", data_type=wcd.DataType.TEXT),
                              wcd.Property(name="price", data_type=wcd.DataType.NUMBER, skip_vectorization=True),
                          ]
                          )

In [6]:
collection = client.collections.get(collection_name)

In [4]:
import tqdm.notebook as tqdm
from datasets import load_dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full", trust_remote_code=True, streaming=True)
with client.batch.fixed_size(batch_size=100, concurrent_requests=50, consistency_level=wcd.ConsistencyLevel.ONE) as batch:
    for item in tqdm.tqdm(iter(dataset)):
        dict = {
            "title": item["title"],
            "category": item["main_category"],
            "description": "\n".join(item["description"]),
            "price": -1 if item["price"] == "None" else float(item["price"]) ,
        }
        batch.add_object(collection=collection_name, properties=dict, uuid=weaviate.util.generate_uuid5(dict["title"]))
        if batch.number_errors > 1:
            break
    if len(client.batch.failed_objects) > 0:
        print("Failed objects:")
        print(client.batch.failed_objects)
        breakpoint()

0it [00:00, ?it/s]

In [7]:
from langchain_core.tools import tool
from weaviate.classes.query import Filter

@tool(name_or_callable="weaviate_search")
def weaviate_search(query: str) -> str:
    """search for products in database

    Args:
        query: query to search for
    """
    response = collection.query.near_text(query, limit=1)
    if len(response) == 0:
        return "No product found"
    return response.objects[0]


@tool(name_or_callable="weaviate_search_with_price_filter")
def weaviate_search_with_price_filter(query: str, maximum_price: int = 9999, minimum_price = 0) -> str:
    """search for products in database and filter results by price

    Args:
        query: query to search for
        maximum_price: maximum price to search for, if not specified, defaults to 9999
        minimum_price: minimum price to search for, if not specified, defaults to 0
    """

    response = collection.query.near_text(query, limit=1, filters=Filter.all_of([
        Filter.by_property("price").greater_than(minimum_price),
        Filter.by_property("price").less_than(maximum_price),
    ]))
    if len(response) == 0:
        return "No product found"
    return response.objects[0]




In [15]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

model = ChatOllama(model="llama3.2")
tools = [weaviate_search, weaviate_search_with_price_filter]

model_with_tools = model.bind_tools(tools)

In [16]:
response = model_with_tools.invoke([HumanMessage(content="Blue lipstick")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'weaviate_search', 'args': {'query': 'blue lipstick'}, 'id': '354f0cec-10bd-45d7-88ba-141240b7cd86', 'type': 'tool_call'}]


In [19]:
response = model_with_tools.invoke([HumanMessage(content="Blue lipstick below 10$")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'weaviate_search_with_price_filter', 'args': {'maximum_price': 10, 'minimum_price': 0, 'query': 'blue lipstick'}, 'id': 'f3206ae0-ccd8-4fa2-80cc-3c59760d83af', 'type': 'tool_call'}]


In [21]:
from typing import Literal
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

tools = [weaviate_search, weaviate_search_with_price_filter]
tool_node = ToolNode(tools)
model = ChatOllama(model="llama3.2")

def should_continue(state: MessagesState) -> Literal["tools", END]:
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')
checkpointer = MemorySaver()

app = workflow.compile(checkpointer=checkpointer)

final_state = app.invoke(
    {"messages": [HumanMessage(content="Blue lipstick below 10$")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'Here are some blue lipstick options under $10:\n\n1. **Sephora Collection Blue Velvet Lipstick**: This shade is a vibrant, blue-based pink that\'s perfect for making a statement. ($8)\n2. **NYX Soft Matte Lip Cream in "Fiji"**: This blue-pink color has a soft, matte finish and is easy to wear. ($6)\n3. **L\'Oreal Paris Infallible Blue Lipstick**: This long-lasting lipstick comes in a range of blue shades, including this vibrant blue. ($9.99)\n4. **Milani Color Statement Blue Velvet Lipstick**: Another affordable option from Milani, this shade is a rich, blue-based pink. ($8.50)\n5. **Maybelline SuperStay Matte Ink Blue Lipstick**: This waterproof lipstick comes in several blue shades and has a comfortable, matte finish. ($9.99)\n\nPlease note that prices may vary depending on the location and retailer.\n\nWould you like me to suggest more options or provide information about a specific brand?'